In [1]:
import pandas as pd
import datetime
import mysql.connector
from mysql.connector import errorcode

In [2]:
schedulers_df = pd.read_csv('schedulers.csv', encoding='cp1251', sep=';')
schedulers_df.head(10)

,ФИО,Расписание,Дата начала расписания,Дата окончания расписания
0,Поставщик 1,дддвсвнн,01.01.2019 0:00,10.01.2019 0:00
1,Поставщик 1,ннвннв,11.01.2019 0:00,15.01.2019 0:00
2,Поставщик 1,св,16.01.2019 0:00,20.01.2019 0:00
3,Поставщик 2,свсвсв,01.01.2019 0:00,07.01.2019 0:00
4,Поставщик 2,днвсв,08.01.2019 0:00,14.01.2019 0:00
5,Поставщик 2,ннддвсв,15.01.2019 0:00,31.12.9999 0:00
6,Поставщик 3,нвнвнв,01.01.2019 0:00,01.02.2019 0:00
7,Поставщик 3,двдвдвдв,02.02.2019 0:00,31.12.9999 0:00


In [3]:
schedulers_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 4 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   ФИО                        8 non-null      object
 1   Расписание                 8 non-null      object
 2   Дата начала расписания     8 non-null      object
 3   Дата окончания расписания  8 non-null      object
dtypes: object(4)
memory usage: 384.0+ bytes


In [4]:
MAX_PANDAS_DATETIME_STR = '31.12.2220 0:00'

In [5]:
schedulers_df = schedulers_df.replace('31.12.9999 0:00', '31.12.2220 0:00')

In [6]:
schedulers_df['Дата начала расписания'] = pd.to_datetime(schedulers_df['Дата начала расписания'], format='%d.%m.%Y %H:%M')
schedulers_df['Дата окончания расписания'] = pd.to_datetime(schedulers_df['Дата окончания расписания'], format='%d.%m.%Y %H:%M')

In [7]:
schedulers_df

,ФИО,Расписание,Дата начала расписания,Дата окончания расписания
0,Поставщик 1,дддвсвнн,2019-01-01,2019-01-10
1,Поставщик 1,ннвннв,2019-01-11,2019-01-15
2,Поставщик 1,св,2019-01-16,2019-01-20
3,Поставщик 2,свсвсв,2019-01-01,2019-01-07
4,Поставщик 2,днвсв,2019-01-08,2019-01-14
5,Поставщик 2,ннддвсв,2019-01-15,2220-12-31
6,Поставщик 3,нвнвнв,2019-01-01,2019-02-01
7,Поставщик 3,двдвдвдв,2019-02-02,2220-12-31


In [8]:
schedulers_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 4 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   ФИО                        8 non-null      object        
 1   Расписание                 8 non-null      object        
 2   Дата начала расписания     8 non-null      datetime64[ns]
 3   Дата окончания расписания  8 non-null      datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 384.0+ bytes


# Задание 1

In [9]:
# Change for your config
user = 'root'
password = open('C://Users/Vlad Molchanov/Desktop/db_password.txt', 'r').readline()
host = 'localhost'

In [10]:
cnx = mysql.connector.connect(user=user, 
                              password=password,
                              host=host)
cursor = cnx.cursor()

In [11]:
DB_NAME = 'business_python'

TABLES = {}

TABLES['t_contractor'] = (
    "CREATE TABLE t_contractor ("
    "id INT NOT NULL PRIMARY KEY AUTO_INCREMENT,"
    "`name` VARCHAR(45) NOT NULL UNIQUE"
    ")" )

TABLES['t_schedule'] = (
    "CREATE TABLE t_schedule ("
    "id INT NOT NULL PRIMARY KEY AUTO_INCREMENT,"
    "`type` VARCHAR(15) NOT NULL UNIQUE"
    ")" )

TABLES['t_contractor_scheduler'] = (
    "CREATE TABLE t_contractor_scheduler ("
    "contractor_id INT NOT NULL,"
    "schedule_id INT NOT NULL,"
    "date_begin DATETIME NOT NULL,"
    "date_end DATETIME NOT NULL,"
    "CONSTRAINT check_date_begin CHECK" 
    "(DATEDIFF(date_end, date_begin) > 0),"
    "FOREIGN KEY (contractor_id)  REFERENCES t_contractor (id) ON DELETE CASCADE,"
    "PRIMARY KEY (contractor_id, date_begin)"
    ")" )

In [12]:
def create_database(cursor):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(DB_NAME))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)

try:
    cursor.execute("USE {}".format(DB_NAME))
    
except mysql.connector.Error as err:
    print("Database {} does not exists.".format(DB_NAME))
    
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cursor)
        print("Database {} created successfully.".format(DB_NAME))
        cnx.database = DB_NAME
        
    else:
        print(err)
        exit(1)

Database business_python does not exists.
Database business_python created successfully.


In [13]:
for table_name in TABLES:
    table_description = TABLES[table_name]
    try:
        print("Creating table {}: ".format(table_name), end='')
        cursor.execute(table_description)
        
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    else:
        print("OK")

Creating table t_contractor: OK
Creating table t_schedule: OK
Creating table t_contractor_scheduler: OK


In [14]:
add_contractor_query = ("INSERT INTO t_contractor "
                        "(name) "
                        "VALUES (%s)")

add_schedule_query = ("INSERT INTO t_schedule "
                      "(type) "
                      "VALUES (%s)")

add_contractor_scheduler_query = ("INSERT INTO t_contractor_scheduler "
                                  "(contractor_id, schedule_id, date_begin, date_end) "
                                  "VALUES (%s, %s, %s, %s)")

In [15]:
select_contractor_id_query = ("SELECT id FROM t_contractor "
                              "WHERE name = %s")

select_schedule_id_query = ("SELECT id FROM t_schedule "
                            "WHERE type = %s")

In [16]:
def try_add_and_get_id(add_query, select_id_query, data):
    try:
        cursor.execute(add_query, data)
        return cursor.lastrowid
        
    except:
        cursor.execute(select_id_query, data)
        return cursor.fetchone()[0]
    
def check_pandas_date(pandas_date):
    if pandas_date == pd.to_datetime(MAX_PANDAS_DATETIME_STR):
        pandas_date = datetime.datetime(9999, 12, 31, 0, 0)
    
    return pandas_date

In [17]:
def insert_data_into_database(df):
    for row in schedulers_df.iterrows():
        row_data = row[1]
        
        contractor_name = row_data['ФИО']
        schedule_type = row_data['Расписание']
        date_begin = row_data['Дата начала расписания']
        date_end = row_data['Дата окончания расписания']
        
        data_contractor = (contractor_name,)
        data_schedule   = (schedule_type,)
        
        contractor_id = -1
        schedule_id   = -1
        
        contractor_id = try_add_and_get_id(add_contractor_query, 
                                           select_contractor_id_query, 
                                           data_contractor)
        
        schedule_id   = try_add_and_get_id(add_schedule_query,
                                           select_schedule_id_query,
                                           data_schedule)
        
        date_begin = check_pandas_date(date_begin)
        date_end   = check_pandas_date(date_end)
        
        data_contractor_scheduler = (contractor_id, schedule_id, date_begin, date_end,)
        cursor.execute(add_contractor_scheduler_query, data_contractor_scheduler)
    
    cnx.commit()

In [18]:
insert_data_into_database(schedulers_df)

# Задание 2

In [19]:
create_t_contractor_work_day = (
    "CREATE TABLE t_contractor_work_day ( "
    "id INT NOT NULL PRIMARY KEY AUTO_INCREMENT,"
    "`name` VARCHAR(45) NOT NULL,"
    "date_begin DATETIME NOT NULL,"
    "date_end DATETIME NOT NULL"
    ");")

In [20]:
cursor.execute(create_t_contractor_work_day)

# Задание 3

In [21]:
work_modes = {'д': [datetime.time(hour=8), datetime.timedelta(hours=12)], 
              'н': [datetime.time(hour=20), datetime.timedelta(hours=12)],
              'с': [datetime.time(hour=8), datetime.timedelta(hours=24)]}

In [22]:
add_work_day_query = ("INSERT INTO t_contractor_work_day "
                      "(`name`, date_begin, date_end)"
                      "VALUES (%s, %s, %s)" )

In [23]:
def add_contractor_work_days(date_start, date_finish):
    for row in schedulers_df.iterrows():
        row_data = row[1]
        
        contractor_name = row_data['ФИО']
        schedule_type = row_data['Расписание']
        date_begin = row_data['Дата начала расписания']
        date_end = row_data['Дата окончания расписания']
        
        date_from = max(date_start, date_begin)
        date_until = min(date_finish, date_end)
        
        current_date = date_from
        counter = 0
        
        while current_date <= date_until:
            work_mode = schedule_type[counter % len(schedule_type)]
            
            if work_mode != 'в':
                work_datetime_begin = datetime.datetime.combine(current_date.date(),
                                                                work_modes[work_mode][0])
                
                data_work_day = (contractor_name, 
                                 work_datetime_begin,
                                 work_datetime_begin + work_modes[work_mode][1],)
                
                cursor.execute(add_work_day_query, data_work_day)
                
            current_date += datetime.timedelta(days=1)
            counter += 1
            
            if current_date > date_until:
                break
        
    cnx.commit()

In [24]:
add_contractor_work_days(datetime.datetime(2019, 1, 1, 0, 0), datetime.datetime(2019, 3, 2, 0, 0))

# Задание 4

In [25]:
contractor_work_day_df = pd.read_sql("SELECT * FROM t_contractor_work_day", cnx)

### Выборка содержащая сколько рабочих дней было у каждого поставщика

In [26]:
contractor_work_day_count = contractor_work_day_df.groupby('name').count().reset_index()[['name', 'id']]
contractor_work_day_count.columns = ['name', 'count']

In [27]:
contractor_work_day_count

,name,count
0,Поставщик 1,15
1,Поставщик 2,43
2,Поставщик 3,31


### Выборка поставщиков, у которых было больше 10 рабочих дней за январь 2019 года

In [28]:
contractors_worked_jan_2019 = contractor_work_day_df[(contractor_work_day_df['date_begin'].dt.month == 1) &
                                                     (contractor_work_day_df['date_begin'].dt.year == 2019)]

In [29]:
(contractors_worked_jan_2019.groupby('name').count() > 10).reset_index()['name']

0    Поставщик 1
1    Поставщик 2
2    Поставщик 3
Name: name, dtype: object

### Выборка поставщиков, кто работал 14, 15 и 16 января 2019 года

In [30]:
contractors_worked_jan_2019[(contractors_worked_jan_2019['date_begin'].dt.day == 14) |
                            (contractors_worked_jan_2019['date_begin'].dt.day == 15) |
                            (contractors_worked_jan_2019['date_begin'].dt.day == 16)]['name'].unique()

array(['Поставщик 1', 'Поставщик 2', 'Поставщик 3'], dtype=object)